In [2]:
import numpy as np
import preprocessing as prp
import configuration as cfg
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

FILE_NAME = 'data2skl_high.mat'
prep = prp.Preprocessing(FILE_NAME)
vois = xrange(prep.voidata.chunks[0]-1)
subs = xrange(len(cfg.Params().sublist))

In [29]:
def voi_analysis(vois, subs):
    for voi in vois:
        print("process voi: %0.0f" % voi)
        sub_data_generator = prep.get_sub_data(voi)
        yield zip(*sub_analysis(subs, sub_data_generator))

def sub_analysis(subs, sub_data_generator):
    for sub in subs:
        X, y, _, _, pipe = sub_data_generator.next()
        yield np.mean(X, axis=1), y, pipe

X, y, pipe = zip(*voi_analysis(vois, subs))

process voi: 0
process voi: 1
process voi: 2
process voi: 3
process voi: 4
process voi: 5
process voi: 6
process voi: 7
process voi: 8
process voi: 9
process voi: 10
process voi: 11
process voi: 12
process voi: 13


In [30]:
# concatinate VOIs
X_new = []
y_new = []
for sub in subs:
    X_sub = []
    y_sub = []
    for voi in vois:
        X_sub.append(X[voi][sub])
        if voi==0:
            y_sub.append(y[0][sub])
    X_new.append(np.asarray(X_sub).T)
    y_new.append(y_sub)

In [51]:
# run classification:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for sub in subs:
    X_tr, X_te, y_tr, y_te = prep.stratified_split(X_new[sub], y_new[sub][0], 0)
    # train on nmsk:
    algo, algo_params, score = prep.params.model
    clf = GridSearchCV(algo, algo_params, cv=3, scoring=score, n_jobs=-1, verbose=0)
    clf.fit(scaler.fit_transform(X_tr), y_tr)
    # test on msk:
    y_true, y_pred = y_te, clf.predict(scaler.transform(X_te))
    print clf.best_score_, accuracy_score(y_true, y_pred)

0.388888888889 0.666666666667
0.75 0.333333333333
0.333333333333 0.333333333333
0.5 0.222222222222
0.5 0.222222222222
0.555555555556 0.555555555556
0.388888888889 0.333333333333
0.555555555556 0.444444444444
0.611111111111 0.111111111111
0.388888888889 0.222222222222
0.555555555556 0.444444444444
0.555555555556 0.333333333333
0.333333333333 0.333333333333
0.666666666667 0.333333333333
0.444444444444 0.333333333333
